In [1]:
#!/usr/bin/env python

This code test the Nystrom method<br>
Given a symmetrick kernel matrix $K \in \mathbb{R}^{n \times n}$ and a submatrix L<br>
$$K = \begin{bmatrix} A & B^T \\ B & C \end{bmatrix}, \quad L = \begin{bmatrix} A \\ B \end{bmatrix}$$<br>
We define the eigenvalues of matrix $A \in \mathbb{R}^{q \times q}$ (where $q << n$) as <br>
$$\sigma(A) = \{\sigma_1, \sigma_2, ..., \sigma_q\}$$<br>
We define the eigenvectors of matrix $A$ as <br>
$$V(A) = \{v_1, v_2, ..., v_q\}$$

The Nystrom method allows us to compute <br>
1. The Entire $K$ matrix using $L$<br>
2. The eigenvectors of $K$ using the eigenvectors of $A$<br>
3. The inverse of $K$

The Nystrom method explained<br>
1. We randomly sample q samples and use that to form the top left matrix A<br>
2. We find $\sigma(A)$ and $V(A)$ <br>
3. We use the r most dominant eigenvectors of A to approximate the eigenfunctions $\phi$<br>
	If we define the feature map <br>
	$$\Psi_q = \begin{bmatrix} \psi(x_1)^T \\ \psi(x_2)^T \\ .. \\ \psi(x_q)^T \end{bmatrix}$$<br>
	Then the relationship between the eigenfunction $\phi_i$ and the eigenvector $v_i$ is<br>
	$$\phi_i = \frac{1}{\sqrt{\sigma_i}} \Psi_q^T v_i$$<br>
4. Once we approximated the eigenfunction, we can use it to approximate $K$ via the following derivation<br>
$$K = \Phi \Phi^T$$<br>
$$K = \begin{bmatrix}  \phi_1(x_1) & \phi_2(x_1) & .. & \phi_r(x_1)\\  \phi_1(x_2) & \phi_2(x_2) & .. & \phi_r(x_2)\\ ... & ... & ... & ...\\ \phi_1(x_n) & \phi_2(x_n) & .. & \phi_r(x_n) \end{bmatrix} \begin{bmatrix}  \phi_1(x_1) & \phi_2(x_1) & .. & \phi_r(x_1)\\  \phi_1(x_2) & \phi_2(x_2) & .. & \phi_r(x_2)\\ ... & ... & ... & ...\\ \phi_1(x_n) & \phi_2(x_n) & .. & \phi_r(x_n) \end{bmatrix}^T $$<br>
$$K = \begin{bmatrix} \frac{1}{\sqrt{\sigma_1}} \Psi_n \Psi_q^T v_1  & \frac{2}{\sqrt{\sigma_2}} \Psi_n \Psi_q^T v_2 & ... & \frac{1}{\sqrt{\sigma_r}} \Psi_n \Psi_q^T v_r\end{bmatrix} \begin{bmatrix}  \phi_1(x_1) & \phi_2(x_1) & .. & \phi_r(x_1)\\  \phi_1(x_2) & \phi_2(x_2) & .. & \phi_r(x_2)\\ ... & ... & ... & ...\\ \phi_1(x_n) & \phi_2(x_n) & .. & \phi_r(x_n) \end{bmatrix}^T $$<br>
$$K =  (\Psi_n \Psi_q^T V \Sigma) (\Psi_n \Psi_q^T V \Sigma)^T$$<br>
where<br>
$$\Sigma = \begin{bmatrix} \frac{1}{\sqrt{\sigma_1}} & 0 & 0 & ... \\ 0 &  \frac{1}{\sqrt{\sigma_2}} & 0 & ... \\ 0 &  0 & \frac{1}{\sqrt{\sigma_2}} & ... \\ ... &  ... & ... & ... \end{bmatrix} $$<br>
Therefore we have<br>
$$K =  (LV\Sigma) (\Sigma V^TL^T)$$<br>
$$K =  \Phi \Phi^T$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
from sklearn.utils import shuffle
from sklearn.kernel_approximation import Nystroem
from tools import *

Initialize all the setting

In [3]:
X = csv_load('../../dataset/wine.csv', shuffle_samples=True)
q = 30				# size of submatrix A
n = X.shape[0]		# number of total samples
γ = get_rbf_γ(X)	# γ used for the gaussian kerenl

Use Nystrom to approximate the kernel

In [4]:
Xa = X[0:q, :]	# A will come from Xa
L = sklearn.metrics.pairwise.rbf_kernel(X, Y=Xa, gamma=γ)
A = L[0:q,:]
[σs,V] = np.linalg.eig(A)
V = V[:,0:10] # only keeping the largest eigenvectors
Σ = np.diag(1/(np.sqrt(σs[0:10])))
Φ = L.dot(V).dot(Σ)
ǩ = Φ.dot(Φ.T)

Use SKlearn to approximate the kernel

In [5]:
feature_map_nystroem = Nystroem(gamma=γ, random_state=1, n_components=q)
Φ2 = feature_map_nystroem.fit_transform(X)
K2 = Φ2.dot(Φ2.T)

Compute the actual kernel

In [6]:
K = sklearn.metrics.pairwise.rbf_kernel(X, gamma=γ)

Display the results<br>
Notice that even though we only used 30 samples to approximate the entire 178 sample, <br>
we still got really good approximation

In [7]:
a= 100; b= 107
print_two_matrices_side_by_side(K[a:b, a:b], ǩ[a:b, a:b], title1='Actual Kernel', title2='Nystrom Kernel')
print_two_matrices_side_by_side(K[a:b, a:b], ǩ[a:b, a:b], title1='Actual Kernel', title2='Sklearn Nystrom Kernel')

                  Actual Kernel                   	Nystrom Kernel                  
[1.     0.2303 0.0173 0.5854 0.458  0.0432 0.9369]	[0.9996 0.2303 0.0172 0.5853 0.4582 0.0432 0.9368]
[0.2303 1.     0.5249 0.0229 0.8947 0.7298 0.1174]	[0.2303 0.9998 0.5248 0.0229 0.8949 0.7299 0.1175]
[0.0173 0.5249 1.     0.0005 0.2765 0.9427 0.0059]	[0.0172 0.5248 0.9997 0.0005 0.2765 0.9429 0.0058]
[0.5854 0.0229 0.0005 1.     0.0737 0.0019 0.7931]	[0.5853 0.0229 0.0005 0.9997 0.074  0.0019 0.7934]
[0.458  0.8947 0.2765 0.0737 1.     0.4505 0.2744]	[0.4582 0.8949 0.2765 0.074  0.9979 0.4503 0.2736]
[0.0432 0.7298 0.9427 0.0019 0.4505 1.     0.0166]	[0.0432 0.7299 0.9429 0.0019 0.4503 0.9998 0.0166]
[0.9369 0.1174 0.0059 0.7931 0.2744 0.0166 1.    ]	[0.9368 0.1175 0.0058 0.7934 0.2736 0.0166 0.9994]
                                                  	


                  Actual Kernel                   	Sklearn Nystrom Kernel              
[1.     0.2303 0.0173 0.5854 0.458  0.0432 0.9369]	[0.9996 0

In [8]:
avg_error = np.sum(np.absolute(K - ǩ))/(n*n)
avg_error2 = np.sum(np.absolute(K - K2))/(n*n)

In [9]:
jupyter_print('The average absolute error with Nystrom of each element is %f'% avg_error)
jupyter_print('The average absolute error with Sklearn Nystrom of each element is %f'% avg_error2)